In [ ]:
!pip install pandas

In [40]:
import os
import json
import pandas as pd
import traceback

In [6]:
from g4f.client import Client
client = Client()
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a paragraph about Gen AI."
        }
    ]
)

print(completion.choices[0].message.content)

Generative AI, or Gen AI, refers to a class of artificial intelligence systems designed to create content, such as text, images, music, and more, by learning patterns from existing data. These models, like GPT-3 and DALL-E, utilize deep learning techniques to generate new and original outputs that can mimic human creativity. The applications of Gen AI are vast, ranging from enhancing creative processes in art and writing to automating tasks in various industries. As this technology continues to evolve, it raises important discussions about ethics, originality, and the future of human creativity, making it a fascinating area of exploration in the field of AI.


In [7]:
from g4f.client import Client as OpenAI


In [10]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [11]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [66]:
TEMPLATE="""
You are an expert the medical field.Given the symptoms {symptoms}, it is your job to \
create a paragraph of {number} 5 in which you will mention the disease and the potential treatments or medicines.
Make sure to be precise even if you don't know say it because different medicine can kill people.
Ensure to make {number} lines.
"""

In [67]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["number","symptoms"],
    template=TEMPLATE
    )

In [33]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
    )

In [70]:
response_message = quiz_generation_prompt.format(
    number=5,
    symptoms=["Abdominal pain and cramping","Chronic diarrhea","Fatigue"]
)

In [71]:
def completions(response_message):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": response_message}]
    )
    print(response.choices[0].message.content)

In [73]:
completions(response_message)

Based on the symptoms of abdominal pain and cramping, chronic diarrhea, and fatigue, it is possible that the individual may be suffering from inflammatory bowel disease (IBD), such as Crohn's disease or ulcerative colitis. Treatment for IBD typically involves a combination of medication, dietary changes, and lifestyle modifications. Medications such as anti-inflammatory drugs, immunosuppressants, and biologic therapies may be prescribed to manage symptoms and reduce inflammation in the digestive tract. It is important for individuals with IBD to work closely with their healthcare provider to develop a personalized treatment plan that addresses their specific needs and concerns. In some cases, surgery may be necessary to remove damaged portions of the digestive tract. Therefore it is crucial for individuals experiencing these symptoms to seek medical attention promptly for an accurate diagnosis and appropriate treatment recommendations.


In [ ]:
TEMPLATE2="""
You are an expert in medical theorapy. Given a Multiple con {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [49]:
from g4f.client import Client
message=""
client = Client()
response = client.chat.completions.create

PromptTemplate(input_variables=['quiz', 'subject'], input_types={}, partial_variables={}, template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. \nif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\nQuiz_MCQs:\n{quiz}\n\nCheck from an expert English Writer of the above quiz:\n')

In [37]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<g4f.Provider.ChatgptFree...t at 0x0000021D19D0A720>, input_type=ChatgptFree]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<g4f.Provider.ChatgptFree...t at 0x0000021D19D0A720>, input_type=ChatgptFree]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [53]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [54]:
file_path='C:/Users/TOPINFORMATIQUE/mcq_generator/data.txt'

In [55]:
with open(file_path,'r') as file:
    data=file.read()

In [56]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [57]:
NUMBER=5 
SUBJECT="artificial intelligence"
TONE="simple"

In [58]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": data,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Artificial intelligence (AI) refers to the simulation of human intelligence in machines programmed to think, learn, and perform tasks typically requiring human intelligence. AI encompasses a range of technologies, from basic rule-based systems to advanced machine learning models that can analyze large datasets and make autonomous decisions. The ultimate goal of AI is to create systems capable of perceiving, reasoning, learning, and interacting with the world in ways that mimic or even surpass human abilities.
You are an expert MCQ maker.Given the above text, it is your job to create a quiz of 5 multiple choice questions for artificial intelligence students in simple tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make shure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make 5 MCQs
##

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [2]:
!pip install -U g4f[all]

  Using cached g4f-0.3.2.9-py3-none-any.whl.metadata (42 kB)
  Using cached pycryptodome-3.21.0-cp36-abi3-win_amd64.whl.metadata (3.4 kB)
  Using cached curl_cffi-0.7.2-cp38-abi3-win_amd64.whl.metadata (12 kB)
  Using cached browser_cookie3-0.19.1-py3-none-any.whl.metadata (632 bytes)
  Using cached PyExecJS-1.5.1.tar.gz (13 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 762.0 kB/s eta 0:00:05
   ----- ---------------------------------- 0.5/4.0 MB 762.0 kB/s eta 0:00:05
   ------- -------------------------------- 0.8/4.0 MB 798.0 kB/s eta 0:00:05
   ------- -------------

In [6]:
from g4f.client import Client

client = Client()
response = client.images.generate(
  model="gemini",
  prompt="a white siamese cat"
)
image_url = response.data[0].url

ERROR:root:Error in synchronous image generation: This event loop is already running


RuntimeError: Cannot close a running event loop